In [5]:
import regex as re
import pandas as pd

In [6]:
import os 
os.chdir('..')
os.getcwd()

'/home/kekeli/Repos/EaseCHAOS'

In [7]:
df = pd.read_excel("data/data1.xlsx", engine="openpyxl", sheet_name="Monday")

In [8]:
df.head(10)

,"UNIVERSITY OF MINES AND TECHNOLOGY, TARKWA",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SEMESTER ONE 2023/2024 TIME TABLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CLASSROOM,NaN,MONDAY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,1,2,3,4,5,6,7,8,9,10,11
5,NaN,7:00-8:00,8:00-9:00,9:00-10:00,10:00-11:00,11:00-12:00,12:30-1:30,1:30-2:30,2:30-3:30,3:30-4:30,4:30-5:30,5:30-6:30,6:30-7:30
6,LH 1,MR 377\nKONADU,NaN,NaN,NaN,GL 159 (P)\nABDEL-FATAO,NaN,LA 159 (P)\nSITTI,NaN,GM 159 (P)\nABDEL-FATAO,NaN,RP 375 (P)\nFANYIN-MARTIN,NaN
7,LH 2,IS 275\nBUAH,NaN,RP 377 (P)\nBAVOH,NaN,RP 159 (P)\nOCRAN,NaN,IS 273\nARYEH,NaN,MN 1C 159 (P)\nARYEH,NaN,GM 351 (P)\nVUNASE,NaN
8,LH 3,MR 1A 151 (P)\nADDO,NaN,MR 1B 151\nADDO,NaN,MR 1A 153 (P)\nAGYARKO,NaN,MR 1B 153\nAGYARKO,NaN,MR 1B 155 (P)\nANKRAH,NaN,PG 371 (P)\nBEMPONG,NaN
9,LH 5,RP 471\nMETEKU,NaN,RN 277 (P)\nOWUSU,NaN,RP 477 (P)\nBAVOH,NaN,NaN,NaN,PG 471\nBRONI-BEDIAKO,NaN,NaN,NaN


In [9]:
def get_time_row(df: pd.DataFrame) -> pd.Series:
    """Get the time row from the dataframe."""
    for row in df.iterrows():
        if re.match(r"^\d{1,2}:\d{1,2}-\d{1,2}:\d{1,2}$", str(row[1].iloc[1])):
            return row            

In [10]:
def get_daily_table(df: pd.DataFrame, class_pattern: str) -> pd.DataFrame:
    """
    Get the a simplified dataframe of the classes for a given class.

    Parameters
    ----------
    df : pandas.DataFrame
        The dataframe to get the simplified time table from.
        It's a general time table on a single day for all classes.
    class_pattern : str
        The class to search for. E.g. 'EL 3'

    Returns
    -------
    pandas.DataFrame
        The simplified dataframe for only the given class.
    """
    df = df.copy()

    time_row = get_time_row(df)
    new_cols = time_row[1].to_list()
    new_cols.pop(0)
    new_cols.insert(0, "Classroom")
    df.columns = new_cols

    df.set_index("Classroom", inplace=True)

    df = df.iloc[time_row[0]+1:]

    df = df.mask(~df.map(lambda x: bool(re.search(class_pattern, str(x)))))
    df = df.dropna(how='all')

    return df

In [11]:
import xlsxwriter

def save_to_excel(df: pd.DataFrame, filename: str) -> None:
    """
    Save the dataframe to an excel file.

    Parameters
    ----------
    df : pandas.DataFrame
        The dataframe to save.
    filename : str
        The filename to save the dataframe to.
    """
    
    # writer = pd.ExcelWriter(filename, engine='xlsxwriter')
    df.to_excel(filename, index=True, sheet_name='Sheet1')
    
    # workbook = writer.book
    # worksheet = writer.sheets['Sheet1']
    # worksheet.set_column('A:XFD', 55)
    # writer.close()

In [20]:
def get_all_daily_tables(filname: str, class_pattern: str) -> dict:
    """
    Get all the daily tables from an excel file.

    Parameters
    ----------
    filname : str
        The filename of the excel file to get the daily tables from.
    class_pattern : str
        The class to get the daily tables or. E.g. 'EL 3'

    Returns
    -------
    dict
        A dictionary of the daily tables for each class.
    """
    sheets = pd.ExcelFile(filname)
    dfs = pd.read_excel(sheets, sheet_name=sheets.sheet_names)

    return {sheet: get_daily_table(dfs[sheet], class_pattern) for sheet in sheets.sheet_names}

In [21]:
def get_time_table(filname: str, class_pattern: str) -> pd.DataFrame:
    """
    Get the complete time table for a particular class for all days.

    Parameters
    ----------
    filname : str
        The filename of the excel file. This file contains every class with the days as the sheet names. 
    class_pattern : str
        The class to get the complete time table for. E.g. 'EL 3'

    Returns
    -------
    pandas.DataFrame
        The complete time table for the given class.
    """
    daily_tables = get_all_daily_tables(filname, class_pattern)

    
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
    for key, value in daily_tables.items():
        if key.title() in days:
            columns = value.columns
            break
    else:
        raise ValueError(f"No sheet found for any of the days: {days}")

    final_df = pd.DataFrame(
        columns=columns,
        index=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"],
    )

    for day, table in daily_tables.items():
        for period, classes in table.items():
            available_classes = classes.dropna()
            if available_classes.any():
                classrooms = classes[classes.notna()].index
                available_classes = [c.replace("\n", " ") for c in available_classes.values]
                available_classes = [f"{c} ({classrooms[i]})" for i, c in enumerate(available_classes)]
                available_classes = '\n'.join(available_classes)
                final_df.loc[day, period] = available_classes

    return final_df

In [22]:
el_timetable = get_time_table("data/data1.xlsx", "EL 3")

In [23]:
ce_timetable = get_time_table("data/D3.xlsx", "CE 3")
ce_timetable

,7:00-8:00,8:00-9:00,9:00-10:00,10:00-11:00,11:00-12:00,12:00-1:00,NaN,1:30-2:30,2:30-3:30,3:30-4:30,4:30-5:30,5:30-6:30,6:30-7:30
Monday,CE 3A 365 (P) KRAMPAH (SF 1)\nCE 3A 3B 379 ABD...,NaN,"CE 3A, CE 3B 363 BUABENG (Mini Auditorium)",NaN,CE 3B 377 (P) UMARU (SF 1),NaN,NaN,CE 3A 367 (P) ABDEL-FATAO (SF 2),NaN,CE 3B 365 (P) KRAMPAH (FI A3),NaN,CE 3B 367 ABDEL-FATAO (FI A1),NaN
Tuesday,CE 3A 377 UMARU (LH 1),NaN,"CE 3B, CE 3A, RN 361 BUABENG (Mini Auditorium)",NaN,NaN,NaN,NaN,CE 3A 381 (P) EFFAH (FI A2),NaN,CE 3B 381 EFFAH (Comp Eng Lab),NaN,NaN,NaN
Wednesday,CE 3A 367 ABDEL-FATAO (GF 2),NaN,"CE 3A, CE 3B 375 (P) NOFONG (Mini Auditorium)",NaN,NaN,NaN,NaN,CE 3A 377 (P) UMARU (Comp Eng Lab),NaN,NaN,CE 3A 379 (P) ABDEL-FATAO (Mini Auditorium),CE 3B 381 (P) EFFAH (PET),NaN
Thursday,NaN,NaN,"EL 3A, CE 3B, CE 3A, EL 3B, MC 3B, MC 3A 365 K...",NaN,"CE 3B, CE 3A 375 NOFONG (Mini Auditorium)",NaN,NaN,NaN,NaN,"CE 3A, CE 3B 373 (P) NOFONG (Mini Auditorium)",NaN,NaN,NaN
Friday,CE 3A 381 EFFAH (Comp Eng Lab)\nCE 3B 377 UMAR...,NaN,NaN,NaN,NaN,NaN,NaN,"CE 3B, CE 3A 373 NOFONG (Mini Auditorium)",NaN,NaN,MC EL CE 363 (P) BUABENG (VLE),NaN,NaN


In [27]:
save_to_excel(ce_timetable, "extract/output/CE 3.xlsx")

'/home/kekeli/Repos/EaseCHAOS'